In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
from keras.utils import to_categorical
from keras.optimizers import SGD
from sklearn.model_selection import StratifiedKFold, train_test_split
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_auc_score
from keras import regularizers

Using TensorFlow backend.


In [2]:
labels = pd.read_csv('hr_labels.csv')
features = pd.read_csv('hr_features.csv')

In [3]:
assert len(features) == len(labels)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
#y_test = pd.DataFrame(y_train)

In [5]:
y_train = pd.DataFrame(y_train)

In [6]:
y_train[:5]

,0
1097,0
727,0
254,0
1175,0
1341,0


In [51]:
num_classes = 1
input_shape = features.shape[1]
activations = 'tanh'

model = Sequential()
model.add(Dense(input_shape, input_shape = (input_shape,), activation = activations,
                kernel_regularizer = regularizers.l2(.001)))
model.add(Dropout(0.6))
model.add(Dense(64, activation = 'relu', kernel_regularizer = regularizers.l2(.001)))#1024
model.add(Dropout(0.6))
model.add(Dense(64, activation = 'relu', kernel_regularizer = regularizers.l2(.001)))#1024
model.add(Dropout(0.6))
model.add(Dense(1, activation = 'sigmoid'))

In [52]:
checkpoint = ModelCheckpoint(filepath = 'weightsz.hdf5', save_best_only = True, monitor='val_loss') 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=0.0001)
                              #cooldown=100, verbose=1)

In [53]:
model.compile(loss = 'binary_crossentropy', optimizer = SGD(lr=0.0001, momentum = 0.9), metrics = ['accuracy'])

In [54]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 48)                2352      
_________________________________________________________________
dropout_15 (Dropout)         (None, 48)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 64)                3136      
_________________________________________________________________
dropout_16 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_17 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 65        
Total para

In [55]:
history = model.fit(x_train, y_train, batch_size = 32, epochs = 3, validation_split = 0.1, verbose = 1)#, callbacks = [checkpoint, reduce_lr])

Train on 1058 samples, validate on 118 samples
Epoch 1/3
1058/1058 [==============================] - 0s 441us/step - loss: 1.1002 - acc: 0.4036 - val_loss: 0.9312 - val_acc: 0.2373
Epoch 2/3
1058/1058 [==============================] - 0s 98us/step - loss: 1.0451 - acc: 0.4386 - val_loss: 0.8905 - val_acc: 0.3475
Epoch 3/3
1058/1058 [==============================] - 0s 96us/step - loss: 0.9611 - acc: 0.4849 - val_loss: 0.8592 - val_acc: 0.5593


In [56]:
model.evaluate(x_test, y_test)

294/294 [==============================] - 0s 52us/step


[0.86293415633999571, 0.48979591857008381]

In [ ]:
evals = [1 if i > 0.1 else 0 for i in predicts]
cf = confusion_matrix(y_test, evals)
cf

In [ ]:
seed = 42
np.random.seed(seed)
num_classes = 1
input_shape = dummy_scaled_df.shape[1]
activations = 'tanh'
X = np.array(all_scaled)
y = np.array(labels)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
X = np.array(X)
y = np.array(y)
for train, test in kfold.split(X, y):
    
    model = Sequential()
    model.add(Dense(input_shape, input_shape = (input_shape,), activation = activations,
                kernel_regularizer = regularizers.l2(.001)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation = 'relu', kernel_regularizer = regularizers.l2(.001)))#1024
    model.add(Dropout(0.5))
    model.add(Dense(64, activation = 'relu', kernel_regularizer = regularizers.l2(.001)))#1024
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation = 'sigmoid'))
    
    cv_checkpoint = ModelCheckpoint(filepath = 'cv_weights.hdf5', save_best_only = True, monitor='val_loss') 
    model.compile(loss = 'binary_crossentropy', optimizer = SGD(lr=0.0001, momentum = 0.9), metrics = ['accuracy'])
    model.fit(X[train], y[train], epochs = 3000, batch_size = 32, verbose = 0,
              callbacks = [cv_checkpoint], validation_split = 0.1)
    
    model.load_weights('cv_weights.hdf5')
    scores = model.evaluate(X[test], y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))